#### Ensemble - RandomForest & ExtraTree
- 배깅 방식의 앙상블 ==> 중복 랜덤 샘플 + 동일 모델(DT)
    * RandomForest:

- 페이스트 방식의 앙상블 ==> 랜덤 샘플 + 동일 모델(DT)
    * 대표 알고리즘 : ExtraTreeC/R

[목표] 와인 분류 => 0과 1 2개 종류 분류

[1] 모듈 로딩 및 데이터 준비

In [2]:
# 모듈 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
DATA_FILE = '../Data/wine.csv'

wineDF = pd.read_csv(DATA_FILE)

In [4]:
# 데이터 확인
wineDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [5]:
wineDF.head(2)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0


In [6]:
# 타겟 / 라벨의 클래스 분포
wineDF['class'].value_counts()

class
1.0    4898
0.0    1599
Name: count, dtype: int64

In [7]:
wineDF.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


[2] 학습 준비

In [8]:
# 학습용 & 테스트용 데이터셋 분할
from sklearn.model_selection import train_test_split

In [9]:
# 피쳐/독립변수와 타겟/라벨/종속변수 분리

featureDF = wineDF[wineDF.columns[:-1]]
targetSR = wineDF[wineDF.columns[-1]]

print(f'featureDF : {featureDF.shape}  targetSR : {targetSR.shape}')

featureDF : (6497, 3)  targetSR : (6497,)


In [10]:
# 학습용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    test_size = 0.2,
                                                    stratify = targetSR,
                                                    random_state = 1)


In [11]:
print(f'X_Train : {X_train.shape} y_train : {y_train.shape}')
print(f'X_Test : {X_test.shape}  y_test : {y_test.shape}')

X_Train : (5197, 3) y_train : (5197,)
X_Test : (1300, 3)  y_test : (1300,)


[3] 학습 진행

In [12]:
# 학습방법 : 지도학습 > 분류
# 알고리즘 : 앙상블 > 배깅 - RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [13]:
# 인스턴스 생성 => 100개의 내부 DT 모델에서 사용할 데이터셋 생성
#                 random_state 매개변수 설정으로 고정된 데이터셋 생성
#                 ood_score 매개변수 : 샘플 데이터셋 추출 후 남은 데이터셋 검증용으로 사용
lf_model = RandomForestClassifier(random_state = 7,
                                  oob_score = True)

# 학습
lf_model.fit(X_train, y_train)

RandomForestClassifier(oob_score=True, random_state=7)

In [14]:
# 모델 파라미터
print(f"classes_ : {lf_model.classes_}")
print(F"n_classes_ : {lf_model.n_classes_}개")
print()
print(f"feature_names_in_ : {lf_model.feature_names_in_}")
print(F"n_features_in : {lf_model.n_features_in_}개")
print(f"feature_importances_ : {lf_model.feature_importances_}")

classes_ : [0. 1.]
n_classes_ : 2개

feature_names_in_ : ['alcohol' 'sugar' 'pH']
n_features_in : 3개
feature_importances_ : [0.23572103 0.49995154 0.26432743]


In [15]:
# 모델 파라미터
print(f'classes_     : {lf_model.estimator_}')
for est in lf_model.estimators_:
    print(est)

classes_     : DecisionTreeClassifier()
DecisionTreeClassifier(max_features='sqrt', random_state=327741615)
DecisionTreeClassifier(max_features='sqrt', random_state=976413892)
DecisionTreeClassifier(max_features='sqrt', random_state=1202242073)
DecisionTreeClassifier(max_features='sqrt', random_state=1369975286)
DecisionTreeClassifier(max_features='sqrt', random_state=1882953283)
DecisionTreeClassifier(max_features='sqrt', random_state=2053951699)
DecisionTreeClassifier(max_features='sqrt', random_state=959775639)
DecisionTreeClassifier(max_features='sqrt', random_state=1956722279)
DecisionTreeClassifier(max_features='sqrt', random_state=2052949340)
DecisionTreeClassifier(max_features='sqrt', random_state=1322904761)
DecisionTreeClassifier(max_features='sqrt', random_state=165338510)
DecisionTreeClassifier(max_features='sqrt', random_state=1133316631)
DecisionTreeClassifier(max_features='sqrt', random_state=4812360)
DecisionTreeClassifier(max_features='sqrt', random_state=372560217)
De

In [16]:
# print(f"classes_        : {lf_model.estimators_samples_}")

In [17]:
print(f"oob_score_ : {lf_model.oob_score_}")

oob_score_ : 0.89532422551472


[4] 성능 평가

In [18]:
train_score = lf_model.score(X_train, y_train)
test_score = lf_model.score(X_test, y_test)

In [19]:
print(f'train_score : {train_score}  test_score : {test_score}')

train_score : 0.9973061381566288  test_score : 0.9


[5] 튜닝
- RandomizedSearchCV 하이퍼파라미터 최적화 클래스
    * 범위가 넓은 하이퍼 파라미터 설정
    * 지정된 범위에서 지정된 횟수만큼 하이퍼 파라미터를 추출하여 조합 진행

In [20]:
# 모듈 로딩
from sklearn.model_selection import RandomizedSearchCV

In [21]:
# RandomForestClassifier 하이퍼파라미터 설정
params = {'max_depth' : range(2, 16),
          'min_samples_leaf' : range(5, 16),
          'criterion' : ['gini', 'entropy', 'log_loss']}

In [22]:
rf_model = RandomForestClassifier(random_state = 7)

In [23]:
searchCV = RandomizedSearchCV(rf_model,
                              param_distributions = params,
                              n_iter = 50,
                              verbose = 4)

In [24]:
searchCV.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END criterion=gini, max_depth=14, min_samples_leaf=10;, score=0.876 total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=14, min_samples_leaf=10;, score=0.835 total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=14, min_samples_leaf=10;, score=0.881 total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=14, min_samples_leaf=10;, score=0.884 total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=14, min_samples_leaf=10;, score=0.871 total time=   0.4s
[CV 1/5] END criterion=entropy, max_depth=12, min_samples_leaf=14;, score=0.875 total time=   0.4s
[CV 2/5] END criterion=entropy, max_depth=12, min_samples_leaf=14;, score=0.839 total time=   0.3s
[CV 3/5] END criterion=entropy, max_depth=12, min_samples_leaf=14;, score=0.876 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=12, min_samples_leaf=14;, score=0.884 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=12, min_samples_leaf

RandomizedSearchCV(estimator=RandomForestClassifier(random_state=7), n_iter=50,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': range(2, 16),
                                        'min_samples_leaf': range(5, 16)},
                   verbose=4)

In [25]:
# 모델 파라미터
print(f'[ searchCV.best_score_ ] {searchCV.best_score_}')
print(f'[ searchCV.best_params_ ] {searchCV.best_params_}')
print(f'[ searchCV.best_estimator_ ] {searchCV.best_estimator_}')

cv_resultDF = pd.DataFrame(searchCV.cv_results_)
cv_resultDF

[ searchCV.best_score_ ] 0.8776253053972015
[ searchCV.best_params_ ] {'min_samples_leaf': 5, 'max_depth': 15, 'criterion': 'log_loss'}
[ searchCV.best_estimator_ ] RandomForestClassifier(criterion='log_loss', max_depth=15, min_samples_leaf=5,
                       random_state=7)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.469159,0.042720,0.026373,0.002041,10,14,gini,"{'min_samples_leaf': 10, 'max_depth': 14, 'cri...",0.875962,0.834615,0.880654,0.883542,0.871030,0.869161,0.017788,19
1,0.450001,0.042436,0.024769,0.001738,14,12,entropy,"{'min_samples_leaf': 14, 'max_depth': 12, 'cri...",0.875000,0.839423,0.875842,0.883542,0.871992,0.869160,0.015349,20
2,0.378866,0.018463,0.024437,0.001817,10,8,gini,"{'min_samples_leaf': 10, 'max_depth': 8, 'crit...",0.873077,0.832692,0.878730,0.879692,0.870067,0.866852,0.017447,26
3,0.281290,0.017228,0.017819,0.001124,9,4,log_loss,"{'min_samples_leaf': 9, 'max_depth': 4, 'crite...",0.828846,0.836538,0.850818,0.856593,0.845043,0.843568,0.009905,45
4,0.433197,0.036608,0.025661,0.003407,10,10,log_loss,"{'min_samples_leaf': 10, 'max_depth': 10, 'cri...",0.878846,0.840385,0.876805,0.885467,0.871992,0.870699,0.015763,13
5,0.365828,0.014064,0.022407,0.001010,13,15,gini,"{'min_samples_leaf': 13, 'max_depth': 15, 'cri...",0.875000,0.831731,0.872955,0.878730,0.871030,0.865889,0.017269,30
6,0.312253,0.006224,0.019277,0.000419,5,6,log_loss,"{'min_samples_leaf': 5, 'max_depth': 6, 'crite...",0.854808,0.829808,0.867180,0.875842,0.856593,0.856846,0.015511,39
7,0.456253,0.015981,0.023056,0.001698,5,14,entropy,"{'min_samples_leaf': 5, 'max_depth': 14, 'crit...",0.886538,0.848077,0.880654,0.890279,0.876805,0.876471,0.014939,2
8,0.396963,0.013577,0.022355,0.001422,6,12,entropy,"{'min_samples_leaf': 6, 'max_depth': 12, 'crit...",0.880769,0.842308,0.877767,0.885467,0.876805,0.872623,0.015455,9
9,0.350782,0.011993,0.020454,0.001465,12,13,entropy,"{'min_samples_leaf': 12, 'max_depth': 13, 'cri...",0.875962,0.836538,0.880654,0.884504,0.873917,0.870315,0.017286,15
